In [1]:
cd /workspace/fast-influence-functions

/workspace/fast-influence-functions


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [3]:
import torch
import transformers
from tqdm import tqdm
from transformers import TrainingArguments
from transformers import default_data_collator
from transformers import AutoModelForSequenceClassification
from typing import Tuple, Dict, Any, Optional, List

from run_experiments import NUM_RETRAINING_EXPERIMENTS
from experiments import constants
from experiments import misc_utils
from experiments.data_utils import glue_output_modes
from experiments.mnli import (
    NUM_DATAPOINTS_TO_REMOVE_CHOICES,
    CORRECT_INDICES,
    INCORRECT_INDICES)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
tokenizer, _ = misc_utils.create_tokenizer_and_model(
    constants.MNLI_MODEL_PATH)

(_,
 mnli_eval_dataset) = misc_utils.create_datasets(
    task_name="mnli",
    tokenizer=tokenizer)

eval_instance_data_loader = misc_utils.get_dataloader(
    dataset=mnli_eval_dataset,
    batch_size=1,
    random=False)

Params Trainable: 14768643
	bert.encoder.layer.10.attention.self.query.weight
	bert.encoder.layer.10.attention.self.query.bias
	bert.encoder.layer.10.attention.self.key.weight
	bert.encoder.layer.10.attention.self.key.bias
	bert.encoder.layer.10.attention.self.value.weight
	bert.encoder.layer.10.attention.self.value.bias
	bert.encoder.layer.10.attention.output.dense.weight
	bert.encoder.layer.10.attention.output.dense.bias
	bert.encoder.layer.10.attention.output.LayerNorm.weight
	bert.encoder.layer.10.attention.output.LayerNorm.bias
	bert.encoder.layer.10.intermediate.dense.weight
	bert.encoder.layer.10.intermediate.dense.bias
	bert.encoder.layer.10.output.dense.weight
	bert.encoder.layer.10.output.dense.bias
	bert.encoder.layer.10.output.LayerNorm.weight
	bert.encoder.layer.10.output.LayerNorm.bias
	bert.encoder.layer.11.attention.self.query.weight
	bert.encoder.layer.11.attention.self.query.bias
	bert.encoder.layer.11.attention.self.key.weight
	bert.encoder.layer.11.attention.self.ke

In [5]:
def get_test_inputs_collections() -> List[int]:
    test_inputs_collections = {}
    for test_index, test_inputs in enumerate(eval_instance_data_loader):
        if test_index in (
                CORRECT_INDICES[:NUM_RETRAINING_EXPERIMENTS] +
                INCORRECT_INDICES[:NUM_RETRAINING_EXPERIMENTS]):

            test_inputs_collections[test_index] = test_inputs

    return test_inputs_collections

# Get the inputs collections
test_inputs_collections = get_test_inputs_collections()

In [6]:
EXPERIMENTS_BASE_DIR = "/export/home/hguo/Experiments/20201019"


def _load_model(
        mode: str,
        example_index: int,
        correct_mode: str,
        tag: str,
        num_data_points_to_remove: int
) -> Tuple[torch.nn.Module, transformers.Trainer]:
    if mode != "original":
        if mode in ["full"]:
            relative_dir = "full"
            hostname = "g0306.ll.unc.edu"

        if mode in ["KNN-1000", "KNN-10000"]:
            relative_dir = "KNN"
            hostname = "g0303.ll.unc.edu"

        if mode in ["random"]:
            relative_dir = "random"
            hostname = "g0310.ll.unc.edu"

        folder_name = (
            f"./retraining-remove-"
            f"{example_index}-"
            f"{correct_mode}-"
            f"{mode}-"
            f"{tag}-"
            f"{num_data_points_to_remove}"
            f".{hostname}"
        )

        model_dir = os.path.join(
            EXPERIMENTS_BASE_DIR,
            relative_dir,
            folder_name,
            "output_dir")
    else:
        if not all([
                example_index is None,
                correct_mode is None,
                tag is None,
                num_data_points_to_remove is None
        ]):
            raise ValueError

        model_dir = constants.MNLI_MODEL_PATH

    model = AutoModelForSequenceClassification.from_pretrained(model_dir)
    model.eval()

    # Most of these arguments are placeholders
    # and are not really used at all, so ignore
    # the exact values of these.
    trainer = transformers.Trainer(
        model=model,
        args=TrainingArguments(
            output_dir="./tmp-output",
            per_device_train_batch_size=128,
            per_device_eval_batch_size=128,
            learning_rate=5e-5,
            logging_steps=100),
    )

    if model.training is True:
        raise ValueError
    
    print(f"Loading from {model_dir}")
    return model, trainer

def _compute_loss(
        trainer: transformers.Trainer,
        model: torch.nn.Module,
        inputs: Dict[str, Any],
) -> torch.Tensor:
    _, _, batch_mean_loss = misc_utils.predict(
        trainer=trainer,
        model=model,
        inputs=inputs)

    return batch_mean_loss


with tqdm(total=None) as pbar:
    loss_collections = []
    for mode in ["full", "KNN-1000", "KNN-10000", "random"]:
        for example_relative_index in range(NUM_RETRAINING_EXPERIMENTS):
            for correct_mode in ["correct", "incorrect"]:
                if correct_mode == "correct":
                    example_index = CORRECT_INDICES[example_relative_index]
                if correct_mode == "incorrect":
                    example_index = INCORRECT_INDICES[example_relative_index]

                if mode in ["full", "KNN-1000", "KNN-10000"]:
                    tags = ["helpful", "harmful"]

                elif mode in ["random"]:
                    tags = ["neutral", "entailment", "contradiction"]

                else:
                    raise ValueError

                for tag in tags:
                    for num_data_points_to_remove in NUM_DATAPOINTS_TO_REMOVE_CHOICES:
                        test_inputs = test_inputs_collections[example_index]

                        model, trainer = _load_model(
                            mode=mode,
                            example_index=example_index,
                            correct_mode=correct_mode,
                            tag=tag,
                            num_data_points_to_remove=num_data_points_to_remove)

                        loss = _compute_loss(
                            trainer=trainer,
                            model=model,
                            inputs=test_inputs)

                        loss_collections.append({
                            "mode": mode,
                            "correct_mode": correct_mode,
                            "example_index": example_index,
                            "tag": tag,
                            "num_data_points_to_remove": num_data_points_to_remove,
                            "loss": loss,
                        })

                        pbar.update(1)

0it [00:00, ?it/s]You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


Loading from /export/home/hguo/Experiments/20201019/full/./retraining-remove-0-correct-full-helpful-1.g0306.ll.unc.edu/output_dir


1it [00:07,  7.45s/it]You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.
2it [00:11,  6.30s/it]

Loading from /export/home/hguo/Experiments/20201019/full/./retraining-remove-0-correct-full-helpful-100.g0306.ll.unc.edu/output_dir


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.
3it [00:14,  5.48s/it]

...

Loading from /export/home/hguo/Experiments/20201019/random/./retraining-remove-15-incorrect-random-contradiction-1.g0310.ll.unc.edu/output_dir


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.
161it [09:20,  3.33s/it]

Loading from /export/home/hguo/Experiments/20201019/random/./retraining-remove-15-incorrect-random-contradiction-100.g0310.ll.unc.edu/output_dir


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.
162it [09:23,  3.48s/it]

Loading from /export/home/hguo/Experiments/20201019/random/./retraining-remove-15-incorrect-random-contradiction-10000.g0310.ll.unc.edu/output_dir


In [7]:
# example_index = 2
# correct_mode = "correct"
# mode = "KNN-10000"
# tag = "helpful"
# num_data_points_to_remove = 100

# model = _load_model(
#     example_index=example_index,
#     correct_mode=correct_mode,
#     mode=mode, tag=tag,
#     num_data_points_to_remove=num_data_points_to_remove)
# inputs = test_inputs_collections[0]

In [8]:
for example_relative_index in range(NUM_RETRAINING_EXPERIMENTS):
    for correct_mode in ["correct", "incorrect"]:
        if correct_mode == "correct":
            example_index = CORRECT_INDICES[example_relative_index]
        if correct_mode == "incorrect":
            example_index = INCORRECT_INDICES[example_relative_index]

        test_inputs = test_inputs_collections[example_index]

        model, trainer = _load_model(
            mode="original",
            example_index=None,
            correct_mode=None,
            tag=None,
            num_data_points_to_remove=None)

        loss = _compute_loss(
            trainer=trainer,
            model=model,
            inputs=test_inputs)

        loss_collections.append({
            "mode": "original",
            "correct_mode": correct_mode,
            "example_index": example_index,
            "tag": None,
            "num_data_points_to_remove": None,
            "loss": loss,
        })

        pbar.update(1)

You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


Loading from /export/home/hguo/Experiments/20200706/


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


Loading from /export/home/hguo/Experiments/20200706/


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


...

Loading from /export/home/hguo/Experiments/20200706/


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


Loading from /export/home/hguo/Experiments/20200706/


You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


Loading from /export/home/hguo/Experiments/20200706/


In [4]:
# import torch
# torch.save(loss_collections, "/export/home/hguo/Experiments/20201102/loss_collections.20201102.pt")
loss_collections = torch.load("/export/home/hguo/Experiments/20201102/loss_collections.20201102.pt")

In [5]:
import pandas as pd
loss_df = pd.DataFrame(loss_collections)

def _get_loss_diff(row):
    loss_diff = loss_df[
        (loss_df["mode"] == "original") &
        (loss_df["correct_mode"] == row["correct_mode"]) &
        (loss_df["example_index"] == row["example_index"])
    ]["loss"].values

    if loss_diff.shape != (1, ):
        raise ValueError

    return loss_diff[0]


loss_df["loss_diff"] = loss_df.apply(lambda row: row["loss"] - _get_loss_diff(row), axis=1)

if not (loss_df[loss_df["mode"] == "original"].loss_diff.values == 0.0).all():
    raise ValueError

# loss_df = loss_df[loss_df.example_index.isin([0, 15, 14])]

In [6]:
new_losses_mean = pd.DataFrame(
    loss_df[loss_df["mode"] != "original"]
    .groupby(["mode", "correct_mode", "tag", "num_data_points_to_remove"])
    ["loss_diff"].mean()
).to_dict()["loss_diff"]

new_losses_std = pd.DataFrame(
    loss_df[loss_df["mode"] != "original"]
    .groupby(["mode", "correct_mode", "tag", "num_data_points_to_remove"])
    ["loss_diff"].std()
).to_dict()["loss_diff"]

original_losses_mean = pd.DataFrame(
    loss_df[loss_df["mode"] == "original"]
    .groupby(["mode", "correct_mode"])
    ["loss_diff"].mean()
).to_dict()["loss_diff"]

original_losses_std = pd.DataFrame(
    loss_df[loss_df["mode"] == "original"]
    .groupby(["mode", "correct_mode"])
    ["loss_diff"].std()
).to_dict()["loss_diff"]

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams.update({"font.size": 15})
figure, axes = plt.subplots(1, 2, sharex=True)


for mode in ["full", "KNN-1000", "KNN-10000", "random"]:
    for axis_index, correct_mode in enumerate(["correct", "incorrect"]):

        if mode in ["full", "KNN-1000", "KNN-10000"]:
            tags = ["helpful", "harmful"]

        elif mode in ["random"]:
            tags = ["neutral", "entailment", "contradiction"]

        else:
            raise ValueError

        for tag in tags:
            Xs = [0]# [0, 1, 100, 10000]
            Ys_mean = [original_losses_mean[("original", correct_mode)]]
            Ys_std_min = [
                original_losses_mean[("original", correct_mode)] -
                original_losses_std[("original", correct_mode)]
            ]
            Ys_std_max = [
                original_losses_mean[("original", correct_mode)] +
                original_losses_std[("original", correct_mode)]
            ]
            for num_data_points_to_remove in NUM_DATAPOINTS_TO_REMOVE_CHOICES:
                Xs.append(num_data_points_to_remove)
                Ys_mean.append(
                    new_losses_mean[(mode, correct_mode, tag, num_data_points_to_remove)]
                )

                Ys_std_min.append(
                    new_losses_mean[(mode, correct_mode, tag, num_data_points_to_remove)] -
                    new_losses_std[(mode, correct_mode, tag, num_data_points_to_remove)]
                )
                Ys_std_max.append(
                    new_losses_mean[(mode, correct_mode, tag, num_data_points_to_remove)] +
                    new_losses_std[(mode, correct_mode, tag, num_data_points_to_remove)]
                )

            axes[axis_index].plot(
                Xs, Ys_mean,
                linestyle="--",
                marker="o",
                label=f"{mode}-{tag}",
                linewidth=2.5,
                markersize=10,
                color={
                    ("full", "helpful"): "#407A78",
                    ("KNN-10000", "helpful"): "#7CA6A0",
                    ("KNN-1000", "helpful"): "#9FBDB6",
                    ("full", "harmful"): "#EBA788",
                    ("KNN-10000", "harmful"): "#FAC8B1",
                    ("KNN-1000", "harmful"): "#F5DAC9",
                    ("random", "neutral"): "lightgrey",
                    ("random", "entailment"): "lightgrey",
                    ("random", "contradiction"): "lightgrey",
                }[(mode, tag)])
            axes[axis_index].legend()
            axes[axis_index].set_xscale("symlog")
            # axes[axis_index].set_yscale("symlog")
            axes[axis_index].set_xlabel("Number of Data-points Removed")
            axes[axis_index].set_ylabel("Change in Loss")
            axes[axis_index].set_title(f"{correct_mode.capitalize()}")
            if correct_mode == "correct":
                axes[axis_index].set_ylim(top=0.25, bottom=-0.25)
            else:
                axes[axis_index].set_ylim(top=0.75, bottom=-0.75)

figure.savefig("./20201102-retraining.pdf")

In [8]:
# for column, correct_mode in enumerate(["correct", "incorrect"]):
#     for mode in ["full", "KNN-1000", "KNN-10000"]:
#         for row, tag in enumerate(["helpful", "harmful"]):

#     for random_label in ["neutral", "entailment", "contradiction"]:
#         Xs = [0, 1, 2, 3]# [0, 1, 100, 10000]
#         Ys_mean = [
#             original_losses_mean[("original", correct_mode)]
#         ] + [
#             losses_mean[("random", correct_mode, random_label, n)] for n in [1, 100, 10000]
#         ]

#         Ys_std_min = [
#             original_losses_mean[("original", correct_mode)] -
#             original_losses_std[("original", correct_mode)]
#         ] + [
#             losses_mean[("random", correct_mode, random_label, n)] -
#             losses_std[("random", correct_mode, random_label, n)]
#             for n in [1, 100, 10000]
#         ]

#         Ys_std_max = [
#             original_losses_mean[("original", correct_mode)] +
#             original_losses_std[("original", correct_mode)]
#         ] + [
#             losses_mean[("random", correct_mode, random_label, n)] +
#             losses_std[("random", correct_mode, random_label, n)]
#             for n in [1, 100, 10000]
#         ]
#         axes[column].plot(Xs, Ys_mean, label=f"{random_label}", linestyle="--")
#         axes[column].legend()
#         if correct_mode == "correct":
#             axes[column].set_ylim(top=0.2, bottom=-0.25)
#         else:
#             axes[column].set_ylim(top=0.25, bottom=-0.5)
#         axes[column].set_title(f"{correct_mode}")

In [145]:
# for mode in ["full", "KNN-1000", "KNN-10000"]:
#     Xs = [0, 1, 100, 10000]
#     Ys = [
#         original_losses_mean[("original", "correct")]
#     ] + [
#         losses_mean[(mode, "incorrect", "harmful", n)] for n in [1, 100, 10000]
#     ]
#     plt.plot(Xs, Ys, label=mode)

# plt.legend()
# plt.xscale("log")